In [3]:
import pandas as pd
from collections import Counter

In [4]:
# data_path = '/Users/ting2liu/data/kiva/kiva_ds_csv/NEW/'
data_path = '../'

In [16]:
df = pd.read_csv(data_path + 'loans.csv', parse_dates = ['posted_time'])

In [14]:
df.columns

Index(['id', 'name', 'original_language', 'original_description',
       'translated_description', 'funded_amount', 'loan_amount', 'status',
       'image_id', 'video_id', 'activity', 'sector', 'use', 'country_code',
       'country_name', 'town', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disbursed_time',
       'funded_time', 'term_in_months', 'lender_count',
       'journal_entries_count', 'bulk_journal_entries_count', 'tags',
       'borrower_names', 'borrower_genders', 'borrower_pictured',
       'repayment_interval', 'distribution_model'],
      dtype='object')

In [5]:
df = pd.read_csv(data_path + 'loans.csv', parse_dates=["POSTED_TIME", "PLANNED_EXPIRATION_TIME",
                               "DISBURSE_TIME", "RAISED_TIME"])

ValueError: 'POSTED_TIME' is not in list

In [7]:
NEW_names = ["LOAN_ID","LOAN_NAME","ORIGINAL_LANGUAGE","DESCRIPTION","DESCRIPTION_TRANSLATED","FUNDED_AMOUNT","LOAN_AMOUNT","STATUS","IMAGE_ID","VIDEO_ID","ACTIVITY_NAME","SECTOR_NAME","LOAN_USE","COUNTRY_CODE","COUNTRY_NAME","TOWN_NAME","CURRENCY_POLICY","CURRENCY_EXCHANGE_COVERAGE_RATE","CURRENCY","PARTNER_ID","POSTED_TIME","PLANNED_EXPIRATION_TIME","DISBURSE_TIME","RAISED_TIME","LENDER_TERM","NUM_LENDERS_TOTAL","NUM_JOURNAL_ENTRIES","NUM_BULK_ENTRIES","TAGS","BORROWER_NAMES","BORROWER_GENDERS","BORROWER_PICTURED","REPAYMENT_INTERVAL","DISTRIBUTION_MODEL"]

In [8]:
OLD_names = ['id','name','original_language','original_description','translated_description','funded_amount','loan_amount','status','image_id','video_id','activity','sector','use','country_code','country_name','town','currency_policy', 'currency_exchange_coverage_rate','currency','partner_id','posted_time','planned_expiration_time','disbursed_time','funded_time','term_in_months','lender_count','journal_entries_count','bulk_journal_entries_count','tags','borrower_names','borrower_genders','borrower_pictured','repayment_interval','distribution_model']

In [9]:
dic = {}
for (k,v) in zip(NEW_names, OLD_names):
    dic[k]=v

In [10]:
df = df.rename(columns=dic)

In [17]:
df.head()

,id,name,original_language,original_description,translated_description,funded_amount,loan_amount,status,image_id,video_id,...,term_in_months,lender_count,journal_entries_count,bulk_journal_entries_count,tags,borrower_names,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,466993,Norma Elizabeth,Spanish,"Norma tiene 38 años de edad, es madre soltera ...",Norma is 38 years old. She is a single mother ...,1000.0,1000.0,funded,1180943.0,NaN,...,20.0,36,5,3,NaN,Norma Elizabeth,female,true,monthly,field_partner
1,468275,Sayda Maria,Spanish,sayda es originaria de la comarca Empalme de B...,Sayda is originally from the Empalme de Boaco ...,225.0,225.0,funded,1185373.0,NaN,...,11.0,9,2,1,NaN,Sayda Maria,female,true,monthly,field_partner
2,468837,Livia,English,Livia is 38 years old and is married to David ...,NaN,725.0,725.0,funded,1180223.0,NaN,...,14.0,25,1,1,NaN,Livia,female,true,monthly,field_partner
3,468882,Eduardo,Spanish,El señor Eduardo de 37 años de edad casado con...,Eduardo is 37 years old. He has been to marrie...,900.0,1475.0,expired,1187226.0,NaN,...,27.0,25,1,1,NaN,Eduardo,male,true,irregular,field_partner
4,469762,Nhanh's Group,English,"Nhanh, age 49, lives in a rural area in the Po...",NaN,1000.0,1000.0,funded,1187927.0,NaN,...,14.0,35,2,1,NaN,"Nhanh, Run","male, male","true, true",monthly,field_partner


In [18]:
df['year'] = df['posted_time'].dt.year #create year column

In [19]:
df.shape

(1312838, 35)

In [20]:
df = df[(df['status'] == 'funded') | (df['status'] == 'expired')]  #only take rows with 'funded' or 'expired' status

In [21]:
df.shape

(1301056, 35)

In [22]:
df.dropna(subset=['borrower_genders'], inplace=True)  # not interested in rows where gender is missing
df.reset_index(drop=True,inplace=True)

In [23]:
df.shape

(1278244, 35)

In [24]:
data = df[['borrower_genders', 'loan_amount', 'sector', 'country_name', 'status', 'year']]  # cols

In [25]:
def genders_groups(x):
    x = x.split(', ')
    if len(x) == 1:
        return x[0]
    else:
        genders = Counter(x).keys()
        if 'female' in genders and 'male' not in genders:
            return 'female_group'
        elif 'female' in genders and 'male' in genders:
            return 'mixed_group'
        else:
            return 'male_group'

In [26]:
data['gender_group'] = data['borrower_genders'].map(genders_groups)

/anaconda2/envs/wordplay/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
data.head()

,borrower_genders,loan_amount,sector,country_name,status,year,gender_group
0,female,1000.0,Housing,El Salvador,funded,2012,female
1,female,225.0,Retail,Nicaragua,funded,2012,female
2,female,725.0,Services,Kenya,funded,2012,female
3,male,1475.0,Agriculture,Bolivia,expired,2012,male
4,"male, male",1000.0,Agriculture,Cambodia,funded,2012,male_group


In [28]:
f = data.groupby(['year', 'gender_group', 'sector'])

In [37]:
final = f['loan_amount'].agg(['mean','size']).reset_index()
final.head()

,year,gender_group,sector,mean,size
0,2006,female,Agriculture,521.607143,140
1,2006,female,Arts,776.428571,35
2,2006,female,Clothing,606.904762,210
3,2006,female,Construction,612.500000,32
4,2006,female,Food,574.211712,666


In [59]:
final['g'] = pd.Categorical(final['gender_group'], categories=['female','female_group','mixed_group'
                                        'male_group','male'], ordered=True)

In [55]:
final['g'][10]

'female'

In [51]:
f = final.sort_values(by='g')

In [60]:
final.head()

,year,gender_group,sector,mean,size,g
807,2017,mixed_group,Wholesale,1300.000000,2,NaN
218,2009,mixed_group,Wholesale,3146.875000,8,NaN
504,2013,mixed_group,Food,3162.581301,1230,NaN
505,2013,mixed_group,Health,1654.385965,171,NaN
506,2013,mixed_group,Housing,2494.207317,82,NaN


In [41]:
final['gender_group'].unique()

array(['mixed_group', 'male_group', 'female', 'female_group', 'male'],
      dtype=object)

In [42]:
final_data = final.query('year != 2018 and year !=2006')

In [43]:
final_data.to_csv('loans_final.csv', index=False)